Plot ATLAS Compact
==================

This ({nb-download}`notebook <Plot-ATLAS-Compact.ipynb>`) demonstrates plotting the global and local solutions from an ATLAS compact model

```{important}
*Need to download tide model prior to running this notebook.*  
```

OTIS format tidal solutions provided by Oregon State University and ESR  
- [http://volkov.oce.orst.edu/tides/region.html](http://volkov.oce.orst.edu/tides/region.html) 

## Python Dependencies
 - [matplotlib: Python 2D plotting library](https://matplotlib.org/)  
 - [numpy: Scientific Computing Tools For Python](https://www.numpy.org)  
 - [pyproj: Python interface to PROJ library](https://pypi.org/project/pyproj/)  
 - [scipy: Scientific Tools for Python](https://www.scipy.org/)  
 - [xarray: N-D labeled arrays and datasets in Python](https://docs.xarray.dev/en/stable/) 

## Program Dependencies

- `io.model.py`: retrieves tide model parameters for named tide models
- `io.OTIS.py`: extract tidal harmonic constants from OTIS tide models  

```{note}
This notebook uses Jupyter widgets to set parameters for calculating the tidal maps.
```

## Load modules

In [ ]:
from __future__ import print_function

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pyTMD.tools
import pyTMD.io

# autoreload
%load_ext autoreload
%autoreload 2

## Set parameters for program

- Model directory  
- Tide model  

In [ ]:
# available model list
model_list = sorted(pyTMD.io.model.ATLAS_compact())
# display widgets for setting directory and model
TMDwidgets = pyTMD.tools.widgets()
TMDwidgets.model.options = model_list
TMDwidgets.model.value = 'TPXO8-atlas'
TMDwidgets.compress.value = False
TMDwidgets.VBox([
    TMDwidgets.directory,
    TMDwidgets.model,
])

## Setup tide model parameters

In [ ]:
# get model parameters
model = pyTMD.io.model(TMDwidgets.directory.value,
   ).from_database(TMDwidgets.model.value)

## Read ATLAS Compact model

In [ ]:
# open ATLAS grid file
dsg, dtg = pyTMD.io.OTIS.open_atlas_grid(model.z.grid_file, use_mmap=True)
ds1 = dsg.tmd.compact.combine_local(dtg)
ds2 = dsg.tmd.compact.refine()

## Plot global and local masks

In [ ]:
%matplotlib widget
# plot the ATLAS mask
fig,ax1 = plt.subplots(num=1, figsize=(8.25,5.25), dpi=120)
dsg.mask.plot(ax=ax1, add_colorbar=False, add_labels=False,
    vmin=0, vmax=1, cmap='gray_r', alpha=0.5)
dsg.local.plot(ax=ax1, add_colorbar=False, add_labels=False,
    vmin=0, vmax=1, cmap='Purples', alpha=0.5)
# no ticks on the x and y axes
ax1.get_xaxis().set_ticks([])
ax1.get_yaxis().set_ticks([])
# stronger linewidth on frame
[i.set_linewidth(2.0) for i in ax1.spines.values()]
# adjust subplot within figure
fig.subplots_adjust(left=0.02,right=0.98,bottom=0.05,top=0.98)
plt.show()

## Plot differences between global and local solutions

In [ ]:
# percent difference between grids
percent = 100.0*(ds1.bathymetry - ds2.bathymetry)/ds1.bathymetry
# plot the percent difference between ATLAS depth
fig,ax2 = plt.subplots(num=2, figsize=(8.25,5.25), dpi=120)
sc = percent.plot(ax=ax2, add_labels=False, add_colorbar=False,
    vmin=-40, vmax=40, cmap='PRGn')
# Add colorbar and adjust size
# pad = distance from main plot axis
# extend = add extension triangles to upper and lower bounds
# options: neither, both, min, max
# shrink = percent size of colorbar
# aspect = lengthXwidth aspect of colorbar
cbar = plt.colorbar(sc, ax=ax2, pad=0.025, extend='both',
    extendfrac=0.0375, orientation='horizontal', shrink=0.925,
    aspect=22, drawedges=False)
# rasterized colorbar to remove lines
cbar.solids.set_rasterized(True)
# Add label to the colorbar
cbar.ax.set_title(f'{model.name} Bathymetry Differences', fontsize=13,
    rotation=0, y=-1.4, va='top')
cbar.ax.set_xlabel('%', fontsize=13, rotation=0, va='center')
cbar.ax.xaxis.set_label_coords(1.065, 0.5)
# ticks lines all the way across
cbar.ax.tick_params(which='both', width=1, length=23,
    labelsize=13, direction='in')
# no ticks on the x and y axes
ax2.get_xaxis().set_ticks([])
ax2.get_yaxis().set_ticks([])
# stronger linewidth on frame
[i.set_linewidth(2.0) for i in ax2.spines.values()]
# adjust subplot within figure
fig.subplots_adjust(left=0.02,right=0.98,bottom=0.05,top=0.98)
plt.show()

In [ ]:
# elevations are returned as (z, localz)
dsh, dth = pyTMD.io.OTIS.open_atlas_elevation(model.z.model_file, use_mmap=True)
ds1 = dsh.tmd.compact.combine_local(dth)
ds2 = dsh.tmd.compact.refine()

In [ ]:
diff = xr.zeros_like(percent)
power = xr.zeros_like(percent)
for i, c in enumerate(ds1.tmd.constituents):
    diff += (ds1[c].real - ds2[c].real)**2 + (ds1[c].imag - ds2[c].imag)**2
    power += ds1[c].real**2 + ds1[c].imag**2

In [ ]:
# calculate the percent difference
percent = 100.0*np.sqrt(diff/power)
# plot the percent between ATLAS tidal amplitudes
fig,ax3 = plt.subplots(num=3, figsize=(8.25,5.25), dpi=120)
sc = percent.plot(ax=ax3, add_labels=False, add_colorbar=False,
    vmin=0, vmax=100, cmap='BuPu')
# Add colorbar and adjust size
# pad = distance from main plot axis
# extend = add extension triangles to upper and lower bounds
# options: neither, both, min, max
# shrink = percent size of colorbar
# aspect = lengthXwidth aspect of colorbar
cbar = plt.colorbar(sc, ax=ax3, pad=0.025, extend='max',
    extendfrac=0.0375, orientation='horizontal', shrink=0.925,
    aspect=22, drawedges=False)
# rasterized colorbar to remove lines
cbar.solids.set_rasterized(True)
# Add label to the colorbar
cbar.ax.set_title(f'{model.name} Tide Height Differences', fontsize=13,
    rotation=0, y=-1.4, va='top')
cbar.ax.set_xlabel('%', fontsize=13, rotation=0, va='center')
cbar.ax.xaxis.set_label_coords(1.065, 0.5)
# ticks lines all the way across
cbar.ax.tick_params(which='both', width=1, length=23,
    labelsize=13, direction='in')
# no ticks on the x and y axes
ax3.get_xaxis().set_ticks([])
ax3.get_yaxis().set_ticks([])
# stronger linewidth on frame
[i.set_linewidth(2.0) for i in ax3.spines.values()]
# adjust subplot within figure
fig.subplots_adjust(left=0.02,right=0.98,bottom=0.05,top=0.98)
plt.show()